使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第二章第二節 設計一次エネルギー消費量）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

from s02_01_designed_primary_energy_consumption import get_f_prim
from s02_01_designed_primary_energy_consumption import get_n_p
from s04_01_general import get_heating_load, get_heating_mode, get_virtual_heating_devices, get_virtual_heatsource, \
    get_E_E_H_d_t, get_E_G_H_d_t, get_E_K_H_d_t, get_E_M_H_d_t, get_E_UT_H_d_t
from s04_01_general import get_cooling_load, get_E_E_C_d_t, get_E_G_C_d_t, get_E_K_C_d_t, get_E_M_C_d_t, get_E_UT_C_d_t
from s05_ventilation import get_E_E_V_d_t
from s06_lighting import get_E_E_L_d_t
from s07_01_hot_water_demand import get_hotwater_load, get_E_E_W_d_t, get_E_G_W_d_t, get_E_K_W_d_t, get_E_M_W_d_t
from s07_01_hot_water_demand import get_default_hw_type, get_virtual_hotwater
from s08_cgs import get_E_G_CG_d_t, get_E_E_CG_gen_d_t, get_L_DHW_d, get_L_HWH_d, get_E_E_CG_self, get_E_E_TU_aux_d_t
from s09_01_pv_system import get_E_E_PV_d_t
from s10_home_appliances_cooking import get_E_E_AP_d_t, get_E_G_AP_d_t, get_E_K_AP_d_t, get_E_M_AP_d_t
from s10_home_appliances_cooking import get_E_E_CC_d_t, get_E_G_CC_d_t, get_E_K_CC_d_t, get_E_M_CC_d_t
from s11_01_external_condition import load_outdoor

# 第二章　単位住戸の一次エネルギー消費量 / Chapter 2 Designed Primary Energy Consumption

## 第二節　設計一次エネルギー消費量 / Section 2 Design primary energy consumption

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．設計一次エネルギー消費量 / Design primary energy consumption

$$
\large 
E_{T}^* = E_{H} + E_{C} + E_{V} + E_{L} + E_{W} - E_{S} + E_{M}
$$

$E_{T}^*$：1 年当たりの設計一次エネルギー消費量(MJ/年) / Design primary energy consumption per year  
$E_{H}$：1 年当たりの暖房設備の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of heating equipment per year  
$E_{C}$：1 年当たりの冷房設備の設計一次エネルギー消費量(MJ/年) / Design of primary cooling energy consumption per year  
$E_{V}$：1 年当たりの機械換気設備の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of mechanical ventilation equipment per year  
$E_{L}$：1 年当たりの照明設備の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of lighting fixtures per year  
$E_{W}$：1 年当たりの給湯設備（コージェネレーション設備を含む）の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of hot water supply equipment (including cogeneration equipment) per year  
$E_{S}$：1 年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量(MJ/年) / Reduction of design primary energy consumption by energy utilization efficiency facility per year  
$E_{M}$：1 年当たりのその他の設計一次エネルギー消費量(MJ/年) / Other design primary energy consumption per year  

In [ ]:
# 1 年当たりの設計一次エネルギー消費量（MJ/年）
def get_E_T_star(E_H, E_C, E_V, E_L, E_W, E_S, E_M):
    return E_H + E_C + E_V + E_L + E_W - E_S + E_M

### 6．暖房設備の設計一次エネルギー消費量 / Design primary energy consumption of heating equipment

$$
\large E_H = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{H,d,t} }}
$$

$E_{H}$：1 年当たりの暖房設備の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of heating equipment per year  
$E_{H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の設計一次エネルギー消費量(MJ/h) / Design primary energy consumption of heating equipment per hour at time $ t $ on date $ d $  

日付$d$の時刻$t$における1 時間当たりの暖房設備の設計一次エネルギー消費量$E_{H,d,t}$は、地域の区分が 8 地域の場合は0とする。

Design of heating equipment per hour at time $ t $ of date $ d $ Primary energy consumption $ E_ {H, d, t} $ is 0 if the area division is 8 areas.

In [ ]:
# 1 年当たりの暖房設備の設計一次エネルギー消費量（MJ/年）
def get_E_H(region, sol_region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, SHC,
            heating_flag_d, L_H_A_d_t, L_T_H_d_t_i):
    if region == 8:
        return 0.0
    elif mode_H is not None:
        E_H_d_t = get_E_H_d_t(region, sol_region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR,
                              mode_OR, CG, SHC, heating_flag_d, L_H_A_d_t, L_T_H_d_t_i) 

        E_H = np.sum(E_H_d_t)

        return E_H
    else:
        return 0.0

$$
\large 
E_{H,d,t} = E_{E,H,d,t} \times f_{prim} \times 10^{-3} + E_{G,H,d,t} + E_{K,H,d,t} + E_{M,H,d,t} + E_{UT,H,d,t}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ of date $ d $  
$f_{prim}$：電気の量 1kWh を熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / Factor to convert 1kWh of electricity into heat quantity (Chapter 2 Section 1 Appendix B)  
$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の設計一次エネルギー消費量(MJ/h) / Design primary energy consumption of heating equipment per hour at time $ t $ on date $ d $  
$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の灯油消費量(MJ/h) / Kerosene consumption of heating equipment per hour at time $ t $ of date $ d $  
$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of heating installation per hour at time $ t $ of date $ d $  
$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h) / Design Primary Energy Consumption Equivalents of Raw Heating Load of Heating Equipment Per Hour at Time $ t $ of Date $ d $   

In [ ]:
# 1 時間当たりの暖房設備の設計一次エネルギー消費量
def get_E_H_d_t(region, sol_region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, SHC,
                heating_flag_d, L_H_A_d_t, L_T_H_d_t_i):
    if region == 8:
        return 0.0
    else:
        # 電気の量 1kWh を熱量に換算する係数
        f_prim = get_f_prim()

        # 暖房設備の消費電力量（kWh/h）
        E_E_H_d_t = get_E_E_H_d_t(region, sol_region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS,
                                  mode_MR, mode_OR, CG, SHC, heating_flag_d, L_H_A_d_t, L_T_H_d_t_i)

        # 暖房設備のガス消費量（MJ/h）
        E_G_H_d_t = get_E_G_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                                  L_T_H_d_t_i)

        # 暖房設備の灯油消費量（MJ/h）
        E_K_H_d_t = get_E_K_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                                  L_T_H_d_t_i)

        # 暖房設備のその他の燃料による一次エネルギー消費量（MJ/h）
        E_M_H_d_t = get_E_M_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i)

        # 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値
        E_UT_H_d_t = get_E_UT_H_d_t(region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                                    CG, L_H_A_d_t, L_T_H_d_t_i)

        return E_E_H_d_t * f_prim / 1000 + E_G_H_d_t + E_K_H_d_t + E_M_H_d_t + E_UT_H_d_

### 7．冷房設備の設計一次エネルギー消費量 / Designing primary energy consumption of cooling system

$$
\large E_C = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{C,d,t} }}
$$

$E_{C}$：1年当たりの冷房設備の設計一次エネルギー消費量(MJ/年) / Design of primary cooling energy consumption per year  
$E_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の設計一次エネルギー消費量(MJ/h) / Designing primary energy consumption of cooling system per hour at time $ t $ of date $ d $  

In [ ]:
# 1 年当たりの冷房設備の設計一次エネルギー消費量
def get_E_C(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t):
    # 1 時間当たりの冷房設備の設計一次エネルギー消費量 (4)
    E_C_d_t = get_E_C_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)

    # 1 年当たりの冷房設備の設計一次エネルギー消費量
    E_C = np.sum(E_C_d_t)

    return E_C

$$
\large 
E_{C,d,t} = E_{E,C,d,t} \times f_{prim} \times 10^{-3} + E_{G,C,d,t} + E_{K,C,d,t} + E_{M,C,d,t}
$$

$E_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の設計一次エネルギー消費量(MJ/h) / Designing primary energy consumption of cooling system per hour at time $ t $ of date $ d $  
$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の消費電力量(kWh/h) / Power consumption of cooling system per hour at time $ t $ of date $ d $  
$E_{G,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のガス消費量(MJ/h) / Gas consumption of cooling system per hour at time $ t $ of date $ d $  
$E_{K,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の灯油消費量(MJ/h) / Kerosene consumption of cooling system per hour at time $ t $ of date $ d $  
$E_{M,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of cooling system per hour at time $ t $ of date $ d $  
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  

In [ ]:
# 1 時間当たりの冷房設備の設計一次エネルギー消費量
def get_E_C_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t):
    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    E_E_C_d_t = get_E_E_C_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)
    E_G_C_d_t = get_E_G_C_d_t()
    E_K_C_d_t = get_E_K_C_d_t()
    E_M_C_d_t = get_E_M_C_d_t()
    E_UT_C_d_t = get_E_UT_C_d_t()

    E_C_d_t = E_E_C_d_t * f_prim / 1000 + E_G_C_d_t + E_K_C_d_t + E_M_C_d_t + E_UT_C_d_t

    return E_C_d_t

### 8．機械換気設備の設計一次エネルギー消費量 / Design primary energy consumption of mechanical ventilation equipment

$$
\large E_V = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,V,d,t} \times f_{prim} \times 10^{-3} }}
$$

$E_{E,V,d,t}$：日付$d$の時刻$t$における1時間当たりの機械換気設備の消費電力量(kWh/h) / Power consumption of mechanical ventilation equipment per hour at time $ t $ of date $ d $  
$E_{V}$：1年当たりの機械換気設備の設計一次エネルギー消費量(MJ/年) / Design of mechanical ventilation equipment per year Primary energy consumption  
$f_{prim}$：電気の量 1kWh を熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / Factor to convert 1kWh of electricity into heat quantity (Chapter 2 Section 1 Appendix B)  

In [ ]:
# 1 年当たりの機械換気設備の設計一次エネルギー消費量
def get_E_V(A_A, V, HEX):
    if V is None:
        return 0.0

    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    n_p = get_n_p(A_A)

    E_E_V_d_t = get_E_E_V_d_t(n_p, A_A, V, HEX)

    E_V = np.sum(E_E_V_d_t) * f_prim / 1000

    return E_V

### 9．照明設備の設計一次エネルギー消費量  / Design primary energy consumption of lighting equipment

$$
\large E_L = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,L,d,t} \times f_{prim} \times 10^{-3} }}
$$

$E_{E,L,d,t}$：日付$d$の時刻$t$における1時間当たりの照明設備の消費電力量(kWh/h) / Power consumption of lighting fixtures per hour at time $ t $ on date $ d $  
$E_{L}$：1年当たりの照明設備の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of lighting fixtures per year  
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  

In [ ]:
# 1 年当たりの照明設備の設計一次エネルギー消費量
def get_E_L(A_A, A_MR, A_OR, L):
    if L is None:
        return 0.0

    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    n_p = get_n_p(A_A)

    E_E_L_d_t = get_E_E_L_d_t(n_p, A_A, A_MR, A_OR, L)

    E_L = np.sum(E_E_L_d_t) * f_prim / 1000
    
    return E_L

### 10．給湯設備及びコージェネレーション設備の設計一次エネルギー消費量 / Design primary energy consumption of hot water supply equipment and cogeneration equipment

給湯設備がコージェネレーション設備でない場合

When the hot water supply system is not a cogeneration facility

$$
\large E_W = {\sum_{d=1}^{365} \sum_{t=0}^{23} {E_{W,d,t} }}
$$

給湯設備がコージェネレーション設備の場合

When the hot water supply system is a cogeneration facility

$$
\large E_W = {\sum_{d=1}^{365} \sum_{t=0}^{23} {E_{CG,d,t} }}
$$

$E_{W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備の設計一次エネルギー消費量(MJ/h) / Designing the primary energy consumption of the water heater per hour at time $ t $ on date $ d $  
$E_{CG,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備の設計一次エネルギー消費量(MJ/h) / Design of cogeneration equipment per hour at time $ t $ on date $ d $ Primary energy consumption  
$E_{W}$：1年当たりの給湯設備（コージェネレーション設備を含む）の設計一次エネルギー消費量(MJ/年) / Design primary energy consumption of hot water supply equipment (including cogeneration equipment) per year  

In [ ]:
# 1 年当たりの給湯設備（コージェネレーション設備を含む）の設計一次エネルギー消費量
def get_E_W(A_A, region, sol_region, HW, SHC, CG, H_A=None, H_MR=None, H_OR=None, H_HS=None, C_A=None, C_MR=None,
            C_OR=None,
            V=None, L=None, A_MR=None, A_OR=None, Q=None, mu_H=None, mu_C=None, NV_MR=None, NV_OR=None, TS=None,
            r_A_ufvnt=None, HEX=None, underfloor_insulation=None, mode_H=None, mode_C=None):
    if HW is None:
        return 0.0, np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365), \
               np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365)

    if HW['hw_type'] != 'コージェネレーションを使用する':
        E_W_d = get_E_W_d(A_A, region, sol_region, HW, SHC, H_HS, H_MR, H_OR, A_MR, A_OR, Q, mu_H, TS, r_A_ufvnt, HEX,
                          underfloor_insulation)

        E_W = np.sum(E_W_d)

        return E_W, np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365), \
               np.zeros(24 * 365), np.zeros(24 * 365), np.zeros(24 * 365)
    else:
        # 1日当たりのコージェネレーション設備の一次エネルギー消費量
        E_G_CG_d_t, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_TU_aux_d_t, E_E_CG_h_d_t, E_G_CG_ded, e_BB_ave, Q_CG_h = \
            calc_E_CG_d_t(A_A, region, sol_region, HW, SHC, CG, H_A, H_MR, H_OR, H_HS, C_A, C_MR, C_OR,
                          V, L, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS,
                                             r_A_ufvnt, HEX, underfloor_insulation, mode_H, mode_C)

        E_CG = np.sum(E_G_CG_d_t)

        return E_CG, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_TU_aux_d_t, E_E_CG_h_d_t, E_G_CG_ded, e_BB_ave, Q_CG_h

#### 10.1 給湯設備の設計一次エネルギー消費量 / Design primary energy consumption of hot water supply system

$$
\large 
E_{W,d,t} = E_{E,W,d,t} \times f_{prim} \times 10^{-3} + E_{G,W,d,t} + E_{K,W,d,t} + E_{M,W,d,t}
$$

$E_{E,W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備の消費電力量(kWh/h) / Power consumption of water heater per hour at time $ t $ on date $ d $    
$E_{G,W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備のガス消費量(MJ/h) / Gas consumption of water heater per hour at time $ t $ on date $ d $  
$E_{K,W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備の灯油消費量(MJ/h) / Consumption of kerosene in water heater per hour at time $ t $ on date $ d $   
$E_{M,W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of water heater per hour at time $ t $ on date $ d $   
$E_{W,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯設備の設計一次エネルギー消費量(MJ/h) / Designing the primary energy consumption of the water heater per hour at time $ t $ on date $ d $  
$f_{prim}$：電気の量 1kWh を熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / Factor to convert 1kWh of electricity into heat quantity (Chapter 2 Section 1 Appendix B)  

In [ ]:
# 1 日当たりの給湯設備の設計一次エネルギー消費量
def get_E_W_d(A_A, region, sol_region, HW, SHC, H_HS=None, H_MR=None, H_OR=None, A_MR=None, A_OR=None, Q=None,
              mu_H=None, TS=None, r_A_ufvnt=None, HEX=None, underfloor_insulation=None):
    # コージェネは対象外
    if HW['hw_type'] == 'コージェネレーションを使用する':
        raise ValueError(HW['hw_type'])

    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    # 想定人数
    n_p = get_n_p(A_A)

    # その他または設置しない場合
    spec_HW = get_virtual_hotwater(region, HW)

    # 温水暖房負荷の計算
    L_HWH = calc_L_HWH(A_A, A_MR, A_OR, HEX, H_HS, H_MR, H_OR, Q, SHC, TS, mu_H, r_A_ufvnt, region, sol_region,
                       underfloor_insulation)

    # 暖房日の計算
    heating_flag_d = calc_heating_flag_d(A_A, A_MR, A_OR, HEX, H_MR, H_OR, Q, SHC, TS, mu_H, r_A_ufvnt, region,
                                         sol_region, underfloor_insulation)

    E_E_W_d = get_E_E_W_d_t(n_p, L_HWH, heating_flag_d, region, sol_region, spec_HW, SHC)
    E_G_W_d = get_E_G_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, spec_HW, SHC)
    E_K_W_d = get_E_K_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, spec_HW, SHC)
    E_M_W_d = get_E_M_W_d_t()

    E_W_d = E_E_W_d * f_prim / 1000 + E_G_W_d + E_K_W_d + E_M_W_d 
    
    return E_W_d

#### 10.2 コージェネレーション設備の設計一次エネルギー消費量 / Design primary energy consumption of cogeneration equipment

$$
\large 
E_{CG,d,t} = E_{G,CG,d,t} + E_{K,CG,d,t}
$$

$E_{CG,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備の設計一次エネルギー消費量(MJ/h) / Design of cogeneration equipment per hour at time $ t $ on date $ d $ Primary energy consumption    
$E_{G,CG,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備のガス消費量(MJ/h) / Gas consumption of cogeneration equipment per hour at time $ t $ of date $ d $    
$E_{K,CG,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備の灯油消費量(MJ/h) / Kerosene consumption of cogeneration equipment per hour at time $ t $ on date $ d $   

In [ ]:
# 1日当たりのコージェネレーション設備の一次エネルギー消費量
def calc_E_CG_d_t(A_A, region, sol_region, HW, SHC, CG, H_A=None, H_MR=None, H_OR=None, H_HS=None, C_A=None, C_MR=None,
                C_OR=None,
                V=None, L=None, A_MR=None, A_OR=None, Q=None, mu_H=None, mu_C=None, NV_MR=None, NV_OR=None, TS=None,
                r_A_ufvnt=None, HEX=None, underfloor_insulation=None, mode_H=None, mode_C=None):
    if HW is None or HW['hw_type'] != 'コージェネレーションを使用する':
        return np.zeros(365), np.zeros(24 * 365), None, None, None, None, None, None

    n_p = get_n_p(A_A)

    # 実質的な暖房機器の仕様を取得
    spec_MR, spec_OR = get_virtual_heating_devices(region, H_MR, H_OR)

    # 実質的な温水暖房機の仕様を取得
    spec_HS = get_virtual_heatsource(region, H_HS)

    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = get_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)

    # 暖房負荷の取得
    L_H_A_d_t, L_T_H_d_t_i, L_dash_H_R_d_t_i = get_heating_load(
        region, sol_region,
        A_A, A_MR, A_OR,
        Q, mu_H, TS, r_A_ufvnt, HEX,
        underfloor_insulation, mode_H,
        spec_MR, spec_OR, mode_MR, mode_OR, SHC)

    # 暖房日の計算
    if SHC is not None and SHC['type'] == '空気集熱式':
        # import section4_1 as H
        # L_T_H_d_t_i, L_dash_H_R_d_t_i = H.get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, None, spec_MR, spec_OR,
        #                                               mode_MR, mode_OR, Q, mu_H, TS, r_A_ufvnt, HEX, SHC,
        #                                               underfloor_insulation)
        #
        from section3_1_heatingday import get_heating_flag_d

        heating_flag_d = get_heating_flag_d(L_dash_H_R_d_t_i)
    else:
        heating_flag_d = None

    # 暖房
    E_E_H_d_t = get_E_E_H_d_t(region, sol_region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                              CG, SHC, heating_flag_d, L_H_A_d_t, L_T_H_d_t_i)
    # 冷房負荷の計算
    L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t = \
        get_cooling_load(region, A_A, A_MR, A_OR, Q, mu_C,
                         NV_MR, NV_OR, r_A_ufvnt, underfloor_insulation,
                         mode_C)

    # 冷房
    E_E_C_d_t = get_E_E_C_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)

    # 換気
    E_E_V_d_t = get_E_E_V_d_t(n_p, A_A, V)

    # 照明
    E_E_L_d_t = get_E_E_L_d_t(n_p, A_A, A_MR, A_OR, L)

    # 家電
    E_E_AP_d_t = get_E_E_AP_d_t(n_p)

    # その他または設置しない場合
    spec_HW = get_virtual_hotwater(region, HW)

    if spec_HW['hw_type'] is not None:
        from section7_1 import get_normalized_bath_function
        # ふろ機能の修正
        bath_function = get_normalized_bath_function(spec_HW['hw_type'], spec_HW['bath_function'])

        # 給湯負荷の生成
        args = {
            'n_p': n_p,
            'region': region,
            'sol_region': sol_region,
            'has_bath': spec_HW['has_bath'],
            'bath_function': bath_function,
            'pipe_diameter': spec_HW['pipe_diameter'],
            'kitchen_watersaving_A': spec_HW['kitchen_watersaving_A'],
            'kitchen_watersaving_C': spec_HW['kitchen_watersaving_C'],
            'shower_watersaving_A': spec_HW['shower_watersaving_A'],
            'shower_watersaving_B': spec_HW['shower_watersaving_B'],
            'washbowl_watersaving_C': spec_HW['washbowl_watersaving_C'],
            'bath_insulation': spec_HW['bath_insulation']
        }
        if SHC is not None:
            if SHC['type'] == '液体集熱式':
                args.update({
                    'type': SHC['type'],
                    'ls_type': SHC['ls_type'],
                    'A_sp': SHC['A_sp'],
                    'P_alpha_sp': SHC['P_alpha_sp'],
                    'P_beta_sp': SHC['P_beta_sp'],
                    'W_tnk_ss': SHC['W_tnk_ss']
                })
            elif SHC['type'] == '空気集熱式':
                args.update({
                    'type': SHC['type'],
                    'hotwater_use': SHC['hotwater_use'],
                    'heating_flag_d': heating_flag_d,
                    'A_col': SHC['A_col'],
                    'P_alpha': SHC['P_alpha'],
                    'P_beta': SHC['P_beta'],
                    'V_fan_P0': SHC['V_fan_P0'],
                    'd0': SHC['d0'],
                    'd1': SHC['d1'],
                    'W_tnk_ass': SHC['W_tnk_ass']
                })
            else:
                raise ValueError(SHC['type'])

        hotwater_load = get_hotwater_load(**args)

        L_dashdash_k_d_t = hotwater_load['L_dashdash_k_d_t']
        L_dashdash_s_d_t = hotwater_load['L_dashdash_s_d_t']
        L_dashdash_w_d_t = hotwater_load['L_dashdash_w_d_t']
        L_dashdash_b1_d_t = hotwater_load['L_dashdash_b1_d_t']
        L_dashdash_b2_d_t = hotwater_load['L_dashdash_b2_d_t']
        L_dashdash_ba1_d_t = hotwater_load['L_dashdash_ba1_d_t']
        L_dashdash_ba2_d_t = hotwater_load['L_dashdash_ba2_d_t']

    # 温水暖房負荷の計算
    if H_HS is not None:
        import section4_1 as H
        from section4_7 import get_L_HWH

        # 実質的な暖房機器の仕様を取得
        spec_MR, spec_OR = H.get_virtual_heating_devices(region, H_MR, H_OR)

        # 暖房方式及び運転方法の区分
        mode_MR, mode_OR = H.get_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)
        spec_HS = H.get_virtual_heatsource(region, H_HS)

        L_T_H_d_t_i, _ = H.get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, None, spec_MR, spec_OR, mode_MR,
                                       mode_OR, Q, mu_H, TS, r_A_ufvnt, HEX, SHC, underfloor_insulation)
    else:
        spec_MR, spec_OR, spec_HS = None, None, None
        mode_MR, mode_OR = None, None
        L_T_H_d_t_i = None

    # 1時間当たりの給湯設備の消費電力量
    E_E_W_d_t = get_E_E_W_d_t(n_p, L_T_H_d_t_i, heating_flag_d, region, sol_region, HW, SHC)

    # 1時間当たりの電力需要
    E_E_dmd_d_t = get_E_E_dmd_d_t(E_E_H_d_t, E_E_C_d_t, E_E_V_d_t, E_E_L_d_t, E_E_W_d_t, E_E_AP_d_t)

    # 1日当たりのコージェネレーション設備の一次エネルギー消費量
    E_G_CG_d_t, E_E_CG_gen_d_t, E_G_CG_ded, E_E_CG_self, Q_CG_h, E_E_TU_aux_d_t, e_BB_ave = \
        get_E_G_CG_d_t(bath_function, CG, E_E_dmd_d_t,
                                        L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t, L_dashdash_b1_d_t,
                                        L_dashdash_b2_d_t,
                                        L_dashdash_ba1_d_t, L_dashdash_ba2_d_t,
                                        spec_HS, spec_MR, spec_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR,
                                        L_T_H_d_t_i)

    # 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    E_E_CG_h_d_t = get_E_E_CG_h_d_t(E_E_CG_gen_d_t, E_E_dmd_d_t, True)

    return E_G_CG_d_t, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_TU_aux_d_t, E_E_CG_h_d_t, E_G_CG_ded, e_BB_ave, Q_CG_h

### 11．その他の設計一次エネルギー消費量 / Other design primary energy consumption

$$
\large E_M = \sum_{d=1}^{365}{\sum_{t=0}^{23}{(E_{AP,d,t} + E_{CC,d,t})}}
$$

$E_{AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の設計一次エネルギー消費量(MJ/h) / Design primary energy consumption of household appliances per hour at time $ t $ on date $ d $   
$E_{CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の設計一次エネルギー消費量(MJ/h) / Designing Primary Energy Consumption of Cooking Per Hour at Time $ t $ on Date $ d $   
$E_{M}$：1 年当たりのその他の設計一次エネルギー消費量(MJ/年) / Other design primary energy consumption per year   

In [ ]:
# 1年当たりのその他の設計一次エネルギー消費量
def get_E_M(A_A):
    # 想定人数
    n_p = get_n_p(A_A)

    # 1 時間当たりの家電の設計一次エネルギー消費量
    E_AP_d_t = get_E_AP_d_t(n_p)

    # 1 時間当たりの調理の設計一次エネルギー消費量
    E_CC_d_t = get_E_CC_d_t(n_p)

    # 1年当たりのその他の設計一次エネルギー消費量
    E_M = np.sum(E_AP_d_t + E_CC_d_t)

    return E_M

#### 11.1 家電の設計一次エネルギー消費量 / Design primary energy consumption of home appliances

$$
\large 
E_{AP,d,t} = E_{E,AP,d,t} \times f_{prim} \times 10^{-3} + E_{G,AP,d,t} + E_{K,AP,d,t} + E_{M,AP,d,t}
$$

$E_{AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の設計一次エネルギー消費量(MJ/h) / Design primary energy consumption of household appliances per hour at time $ t $ on date $ d $    
$E_{E,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の消費電力量(kWh/h) / Power consumption of household appliances per hour at time $ t $ of date $ d $    
$E_{G,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電のガス消費量(MJ/h) / Gas consumption of household appliances per hour at time $ t $ on date $ d $  
$E_{K,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の灯油消費量(MJ/h) / Household appliances kerosene consumption per hour at time $ t $ on date $ d $   
$E_{M,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of household appliances per hour at time $ t $ on date $ d $   
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  

In [ ]:
# 1 時間当たりの家電の設計一次エネルギー消費量
def get_E_AP_d_t(n_p):
    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()
    return get_E_E_AP_d_t(n_p) * f_prim / 1000 + get_E_G_AP_d_t() + get_E_K_AP_d_t() + get_E_M_AP_d_t()

#### 11.2 調理の設計一次エネルギー消費量 / Design primary energy consumption of cooking

$$
\large 
E_{CC,d,t} = E_{E,CC,d,t} \times f_{prim} \times 10^{-3} + E_{G,CC,d,t} + E_{K,CC,d,t} + E_{M,CC,d,t}
$$

$E_{CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の設計一次エネルギー消費量(kWh/h) / Designing Primary Energy Consumption of Cooking Per Hour at Time $ t $ on Date $ d $    
$E_{E,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の消費電力量(kWh/h) / Cooking power consumption per hour at time $ t $ on date $ d $    
$E_{G,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理のガス消費量(MJ/h) / Cooking gas consumption per hour at time $ t $ on date $ d $  
$E_{K,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の灯油消費量(MJ/h) / Cooking kerosene consumption per hour at time $ t $ on date $ d $   
$E_{M,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by other fuels of cooking per hour at time $ t $ on date $ d $   
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  

In [ ]:
# 1 時間当たりの調理の設計一次エネルギー消費量
def get_E_CC_d_t(n_p):
    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    return get_E_E_CC_d_t() * f_prim / 1000 + get_E_G_CC_d_t(n_p) + get_E_K_CC_d_t() + get_E_M_CC_d_t()

### 12．エネルギー利用効率化設備による設計一次エネルギー消費量の削減量 / Reduction of design primary energy consumption by energy efficiency equipment

$$
\large E_S = E_{S,h} + E_{S,sell}
$$

$E_S$：1 年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量(MJ/年) / Reduction of design primary energy consumption by energy utilization efficiency facility per year   
$E_{S,h}$：1 年当たりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る設計一次エネルギー消費量の削減量(MJ/年) / Design primary energy consumption reduction related to the self-consumption among the power generation capacity by the energy utilization efficiency improvement facility per year   
$E_{S,sell}$：1 年当たりのコージェネレーション設備による売電量に係る設計一次エネルギー消費量の控除量(MJ/年) / Deduction of design primary energy consumption related to the amount of electricity sold by cogeneration equipment per year   

In [ ]:
# 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr) (14)
def get_E_S(E_S_h, E_S_sell):
    """
    :param E_S_h: 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr)
    :param E_S_sell: 1年当たりのエネルギー利用効率化設備による発電量のうち自家消費分に係る設計一次エネルギー消費量の削減量 (MJ/yr)
    :return: 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr)
    """
    return E_S_h + E_S_sell

$$
\large E_{S,h} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{(E_{E,PV,h,d,t} + E_{E,CG,h,d,t}) \times f_{prim} \times 10^{-3} }}
$$

$E_{E,PV,h,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量のうちの自家消費分(kWh/h) / Of the amount of power generated by the photovoltaic power generation facility per hour at time $ t $ on date $ d $   
$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/h) / Of the power generated by the cogeneration facility per hour at time $ t $ on date $ d $   
$E_{S,h}$：1 年当たりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る設計一次エネルギー消費量の削減量(MJ/年) / Design primary energy consumption reduction related to the self-consumption among the power generation capacity by the energy utilization efficiency improvement facility per year   
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  

In [ ]:
# 1年当たりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る一次エネルギー消費量の控除量 (MJ/yr) 
def get_E_S_h(E_E_PV_h_d_t, E_E_CG_h_d_t):
    """
    :param E_E_PV_h_d_t: 1時間当たりの太陽光発電設備による発電量のうちの自家消費分 (kWh/h)
    :param E_E_CG_h_d_t: 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    :return: 1年あたりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る一次エネルギー消費量の控除量 (MJ/yr)
    """
    # 電気の量 1kWh を熱量に換算する係数 (kJ/kWh)
    f_prim = get_f_prim()

    # 1年あたりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る一次エネルギー消費量の控除量 (MJ/yr)
    E_S_h = np.sum(E_E_PV_h_d_t + E_E_CG_h_d_t) * f_prim * 1e-3

    return E_S_h

$$
\large E_{S,sell} = E_{G,CG,sell}
$$

$E_{S,sell}$：1 年当たりのコージェネレーション設備による売電量に係る設計一次エネルギー消費量の控除量(MJ/年) / Deduction of design primary energy consumption related to the amount of electricity sold by cogeneration equipment per year   
$E_{G,CG,sell}$：1 年当たりのコージェネレーション設備による売電量に係るガス消費量の控除量(MJ/年) / Deduction of gas consumption amount related to power sales volume by cogeneration facility per year   

In [ ]:
# 1年当たりのコージェネレーション設備の売電量に係る設計一次エネルギー消費量の控除量 (MJ/yr) 
def get_E_S_sell(E_G_CG_sell):
    """
    :param E_G_CG_sell: 1年当たりのコージェネレーション設備の売電量に係る設計一次エネルギー消費量の控除量 (MJ/yr)
    :return: 1年当たりのコージェネレーション設備の売電量に係る設計一次エネルギー消費量の控除量 (MJ/yr)
    """
    return E_G_CG_sell

#### 12.1 太陽光発電設備による発電量のうちの自家消費分 / In-house consumption of the amount of electricity generated by the photovoltaic power generation facility

太陽光発電設備を採用しない場合

When not using solar power generation equipment

$$
\large 
E_{E,PV,h,d,t} = 0
$$

太陽光発電設備を採用する場合

When adopting a solar power generation facility

$$
\large 
E_{E,PV,h,d,t} = min(E_{E,PV,d,t}, E_{E,dmd,d,t} - E_{E,CG,h,d,t})
$$

$E_{E,PV,h,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量のうちの自家消費分(kWh/h) / Of the amount of power generated by the photovoltaic power generation facility per hour at time $ t $ on date $ d $    
$E_{E,PV,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量(kWh/h) / The amount of power generated by the solar power generation facility per hour at the time $ t $ of the date $ d $  
$E_{E,dmd,d,t}$：日付$d$の時刻$t$における1 時間当たりの電力需要(kWh/h) / Power demand per hour at time $ t $ on date $ d $   
$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量(kWh/h) / Amount of power generated by the cogeneration facility per hour at time $ t $ of date $ d $   

In [ ]:
# 1 時間当たりの太陽光発電設備による発電炉湯のうちの自家消費分 (kWh/h)
def get_E_E_PV_h_d_t(E_E_PV_d_t, E_E_dmd_d_t, E_E_CG_h_d_t, has_PV):
    """
    :param E_E_PV_d_t: 1時間当たりの太陽光発電設備による発電量のうちの自家消費分 (kWh/h)
    :param E_E_dmd_d_t: 1時間当たりの太陽光発電設備による発電量 (kWh/h)
    :param E_E_CG_h_d_t: 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    :param has_PV: 太陽光発電設備を採用する場合はTrue
    :return: 1 時間当たりの太陽光発電設備による発電炉湯のうちの自家消費分 (kWh/h)
    """
    if has_PV == False:
        # 太陽光発電設備を採用しない場合
        E_E_PV_h_d_t = np.zeros_like(E_E_PV_d_t)
    else:
        # 太陽光発電設備を採用する場合
        E_E_PV_h_d_t = np.minimum(E_E_PV_d_t, E_E_dmd_d_t - E_E_CG_h_d_t)

    return E_E_PV_h_d_t

#### 12.2 コージェネレーション設備による発電量のうちの自家消費分 / In-house consumption of power generation by cogeneration equipment

$$
\large 
E_{E,CG,h} = \sum_{d=1}^{365} \sum_{t=0}^{23} E_{E,CG,h,d,t}
$$

$E_{E,CG,h}$：1 年当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/年) / In-house consumption of power generated by cogeneration facilities per year    
$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/h) / Of the power generated by the cogeneration facility per hour at time $ t $ on date $ d $  

In [ ]:
# 1年当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/yr)
def get_E_E_CG_h(E_E_CG_h_d_t):
    """
    :param E_E_CG_h_d_t: 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    :return: 1年当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/yr)
    """
    return np.sum(E_E_CG_h_d_t)

コージェネレーション設備を採用しない場合

When not using cogeneration equipment

$$
\large 
E_{E,CG,h,d,t} = 0
$$

コージェネレーション設備を採用する場合

When adopting cogeneration equipment

$$
\large 
E_{E,CG,h,d,t} = min(E_{E,CG,gen,d,t}, E_{E,dmd,d,t})
$$

$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/h) / Of the power generated by the cogeneration facility per hour at time $ t $ on date $ d $  
$E_{E,dmd,d,t}$：日付$d$の時刻$t$における1 時間当たりの電力需要(kWh/h) / Power demand per hour at time $ t $ on date $ d $   
$E_{E,CG,gen,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量(kWh/h) / Amount of power generated by the cogeneration facility per hour at time $ t $ of date $ d $   

In [ ]:
# 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
def get_E_E_CG_h_d_t(E_E_CG_gen_d_t, E_E_dmd_d_t, has_CG):
    """
    :param E_E_CG_gen_d_t: 1時間当たりのコージェネレーション設備による発電のうちの自家消費分 (kWh/h)
    :param E_E_dmd_d_t: 1時間当たりの電力需要 (kWh/h)
    :param has_CG: コージェネレーション設備を採用する場合はTrue
    :return: 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    """
    if has_CG == False:
        # コージェネレーション設備を採用しない場合
        E_E_CG_h_d_t = np.zeros_like(E_E_CG_gen_d_t)
    else:
        # コージェネレーション設備を採用する場合
        E_E_CG_h_d_t = np.minimum(E_E_CG_gen_d_t, E_E_dmd_d_t)
    return E_E_CG_h_d_t

#### 12.3 コージェネレーション設備による売電量に係るガス消費量の控除量 / Deduction amount of gas consumption amount related to power sales amount by cogeneration facility

コージェネレーション設備を採用しない場合

When not using cogeneration equipment

$$
\large 
E_{E,CG,sell} = 0
$$

コージェネレーション設備を採用する場合

When adopting cogeneration equipment

$$
\large 
E_{E,CG,sell} = E_{G,CG,ded} \times \frac{E_{CG,sell}}{E_{CG,sell}+(E_{E,CG,self}+E_{E,CG,h}) \times f_{prim} \times 10^{-3} + Q_{CG,h} \div e_{BB,ave}}
$$

$E_{CG,sell}$：1 年当たりのコージェネレーション設備による売電量（一次エネルギー）(MJ/年) / Amount of power sold by cogeneration facilities per year (primary energy)  
$E_{E,CG,self}$：1 年当たりのコージェネレーション設備による発電量のうちの自己消費分(kWh/年) / Of the amount of power generated by cogeneration facilities per year, for consumption  
$E_{E,CG,h}$：1 年当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/年) / In-house consumption of power generated by cogeneration facilities per year  
$E_{G,CG,sell}$：1 年当たりのコージェネレーション設備による売電量に係るガス消費量の控除量(MJ/年) / Deduction of gas consumption amount related to power sales volume by cogeneration facility per year  
$E_{G,CG,ded}$：1 年当たりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分(MJ/年) / Of the gas consumption of the cogeneration facility per year, deductible for the sale of electricity  
$e_{BB,ave}$：コージェネレーション設備の給湯時のバックアップボイラーの年間平均効率(-) / Annual average efficiency of backup boiler during hot water supply of cogeneration facility  
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity to heat (Chapter 2, Section 1 Appendix B)  
$Q_{CG,h}$：1 年当たりのコージェネレーション設備による製造熱量のうちの自家消費算入分(MJ/年) / Of consumption heat generated by cogeneration facility per year, including consumption by house   

In [ ]:
# 1年当たりのコージェネレーション設備による売電量に係るガス消費量の控除量 (MJ/yr)
def get_E_G_CG_sell(E_CG_sell, E_E_CG_self, E_E_CG_h, E_G_CG_ded, e_BB_ave, Q_CG_h, has_CG):
    """
    :param E_CG_sell: 1年当たりのコージェネレーション設備による売電量(一次エネルギー換算値) (MJ/yr)
    :param E_E_CG_self: 1年当たりのコージェネレーション設備による発電量のうちの自己消費分 (kWh/yr)
    :param E_E_CG_h: 1年当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/yr)
    :param E_G_CG_ded: 1年当たりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分 (MJ/yr)
    :param e_BB_ave: コージェネレーション設備の給湯時のバックアップボイラーの年間平均効率 (-)
    :param Q_CG_h: 1年当たりのコージェネレーション設備による製造熱量のうちの自家消費算入分 (MJ/yr)
    :param has_CG: コージェネレーション設備を採用する場合はTrue
    :return: 1年当たりのコージェネレーション設備による売電量に係るガス消費量の控除量 (MJ/yr)
    """
    if has_CG == False:
        # コージェネレーション設備を採用しない場合
        E_E_CG_d_t = np.zeros_like(E_CG_sell)
    else:
        # 電気の量 1kWh を熱量に換算する係数
        f_prim = get_f_prim()

        # コージェネレーション設備を採用する場合
        denominator = E_CG_sell + (E_E_CG_self + E_E_CG_h) * f_prim * 1e-3 + Q_CG_h / e_BB_ave
        E_E_CG_d_t = E_G_CG_ded * (E_CG_sell / denominator)

    return E_E_CG_d_t

$$
\large E_S = E_{S,h} + E_{S,sell}
$$

$E_S$：1 年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量(MJ/年) / Reduction of design primary energy consumption by energy utilization efficiency facility per year   
$E_{S,h}$：1 年当たりのエネルギー利用効率化設備による発電量のうちの自家消費分に係る設計一次エネルギー消費量の削減量(MJ/年) / Design primary energy consumption reduction related to the self-consumption among the power generation capacity by the energy utilization efficiency improvement facility per year   
$E_{S,sell}$：1 年当たりのコージェネレーション設備による売電量に係る設計一次エネルギー消費量の控除量(MJ/年) / Deduction of design primary energy consumption related to the amount of electricity sold by cogeneration equipment per year   

In [ ]:
# 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr) 
def get_E_S(E_S_h, E_S_sell):
    """
    :param E_S_h: 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr)
    :param E_S_sell: 1年当たりのエネルギー利用効率化設備による発電量のうち自家消費分に係る設計一次エネルギー消費量の削減量 (MJ/yr)
    :return: 1年当たりのエネルギー利用効率化設備による設計一次エネルギー消費量の削減量 (MJ/yr)
    """
    return E_S_h + E_S_sell

#### 12.4 太陽光発電設備による売電量（参考）  / Electricity sales amount by photovoltaic power generation equipment (reference)

$$
\large 
E_{PV,sell} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,PV,sell,d,t} \times f_{prim} \times 10^{-3} }}
$$

$E_{PV,sell}$：1 年当たりの太陽光発電設備による売電量（一次エネルギー）(MJ/年) / The amount of electricity sold by solar power generation facilities per year (primary energy)   
$E_{E,PV,sell,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h) / The amount of electricity sold by the solar power generation facility per hour at time $ t $ on date $ d $ (secondary energy)   
$f_{prim}$：電気の量 1kWh を熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / Factor to convert 1kWh of electricity into heat quantity (Chapter 2 Section 1 Appendix B)  

In [ ]:
# 1 年当たりの太陽光発電設備による売電量（一次エネルギー換算値）
def calc_E_PV_sell():
    """
    :return: 1 年当たりの太陽光発電設備による売電量（一次エネルギー換算値）(MJ/yr)
    """
    # 電気の量 1kWh を熱量に換算する係数 (kJ/kWh)
    f_prim = get_f_prim()

    # 1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h)
    E_E_PV_sell_d_t = calc_E_E_PV_sell_d_t()

    # 1 年当たりの太陽光発電設備による売電量（一次エネルギー換算値）(MJ/yr)
    E_PV_sell = np.sum(E_E_PV_sell_d_t) * f_prim * 1e-3

    return E_PV_sell

$$
\large 
E_{E,PV,sell,d,t} = E_{E,PV,d,t} - E_{E,PV,h,d,t}
$$

$E_{E,PV,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量(kWh/h) / The amount of power generated by the solar power generation facility per hour at the time $ t $ of the date $ d $  
$E_{E,PV,h,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量のうちの自家消費分(kWh/h) / Of the amount of power generated by the photovoltaic power generation facility per hour at time $ t $ on date $ d $   
$E_{E,PV,sell,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h) / The amount of electricity sold by the solar power generation facility per hour at time $ t $ on date $ d $ (secondary energy)  

In [ ]:
# 1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h)
def calc_E_E_PV_sell_d_t():
    """
    :return: 1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h)
    """

    # 太陽光発電設備の発電量 
    E_E_PV_d_t = get_E_E_PV_d_t()

    # 1 時間当たりの太陽光発電設備による発電炉湯のうちの自家消費分 (kWh/h) 
    E_E_PV_h_d_t = get_E_E_PV_h_d_t()

    # 1 時間当たりの太陽光発電設備による売電量（二次エネルギー）(kWh/h) 
    E_E_PV_sell_d_t = E_E_PV_d_t - E_E_PV_h_d_t

    return E_E_PV_sell_d_t

#### 12.5 コージェネレーション設備による売電量（参考） / Sales amount by cogeneration equipment (reference) 

$$
\large 
E_{CG,sell} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,CG,sell,d,t} \times f_{prim} \times 10^{-3} }}
$$

$E_{CG,sell}$：1 年当たりのコージェネレーション設備による売電量（一次エネルギー）(MJ/年) / Amount of power sold by cogeneration facilities per year (primary energy)   
$E_{E,CG,sell,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による売電量（二次エネルギー）(kWh/h) / Amount of power sold by cogeneration equipment per hour at time $ t $ on date $ d $ (secondary energy)   
$f_{prim}$：電気の量1kWhを熱量に換算する係数(kJ/kWh) / A factor to convert 1 kWh of electricity into heat  

In [ ]:
# 1年当たりのコージェネレーション設備による売電量（一次エネルギー換算値）(MJ/yr) 
def get_E_CG_sell(E_E_CG_sell_d_t):
    """
    :param E_E_CG_sell_d_t: 1時間当たりのコージェネレーション設備による売電量(二次エネルギー) (kWh/h)
    :return: 1年当たりのコージェネレーション設備による売電量（一次エネルギー換算値）(MJ/yr)
    """
    # 電気の量 1kWh を熱量に換算する係数 (kJ/kWh)
    f_prim = get_f_prim()

    # 1年当たりのコージェネレーション設備による売電量（一次エネルギー換算値）(MJ/yr) 
    E_CG_sell = np.sum(E_E_CG_sell_d_t) * f_prim * 1e-3

    return E_CG_sell

逆潮流を行わない場合

When not doing reverse power flow

$$
\large 
E_{E,CG,sell,d,t} = 0
$$

逆潮流を行う場合

In case of reverse power flow

$$
\large 
E_{E,CG,sell,d,t} = E_{E,CG,gen,d,t} - E_{E,CG,h,d,t}
$$

$E_{E,CG,gen,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量(kWh/h) / Amount of power generated by the cogeneration facility per hour at time $ t $ of date $ d $   
$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/h) / Of the power generated by the cogeneration facility per hour at time $ t $ on date $ d $   
$E_{E,CG,sell,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による売電量（二次エネルギー）(kWh/h) / Amount of power sold by cogeneration equipment per hour at time $ t $ on date $ d $ (secondary energy)   

In [ ]:
# 1時間当たりのコージェネレーション設備による売電量(二次エネルギー) (kWh/h) 
def get_E_E_CG_sell_d_t(E_E_CG_gen_d_t, E_E_CG_h_d_t, has_CG_reverse):
    """
    :param E_E_CG_gen_d_t: 1時間当たりのコージェネレーション設備による発電量 (kWh/h)
    :param E_E_CG_h_d_t: 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    :param has_CG_reverse: コージェネレーション設備が逆潮流を行う場合はTrue
    :return: 1時間当たりのコージェネレーション設備による売電量(二次エネルギー) (kWh/h)
    """
    if has_CG_reverse == False:
        # 逆潮流を行わない場合 
        E_E_CG_sell_d_t = np.zeros_like(E_E_CG_gen_d_t)
    else:
        # 逆潮流を行う場合
        E_E_CG_sell_d_t = E_E_CG_gen_d_t - E_E_CG_h_d_t
    return E_E_CG_sell_d_t

### 13．設計二次エネルギー消費量 / Design secondary energy consumption

$$
\displaystyle E_E = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,H,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,C,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,V,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,L,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,W,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,AP,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,CC,d,t} }} - \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,PV,h,d,t} }} - \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{E,CG,h,d,t} }}
$$

$E_{E}$：1 年当たりの設計消費電力量(kWh/年) / Design power consumption per year   
$E_{E,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの暖房設備の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ of date $ d $    
$E_{E,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの冷房設備の消費電力量(kWh/h) / Power consumption of cooling system per hour at time $ t $ of date $ d $    
$E_{E,V,d,t}$：日付$d$の時刻$t$における1 時間当たりの機械換気設備の消費電力量(kWh/h) / Power consumption of mechanical ventilation equipment per hour at time $ t $ of date $ d $    
$E_{E,L,d,t}$：日付$d$の時刻$t$における1 時間当たりの照明設備の消費電力量(kWh/h) / Power consumption of lighting fixtures per hour at time $ t $ on date $ d $    
$E_{E,W,d,t}$：日付$d$の時刻$t$における1 時間当たりの給湯設備の消費電力量(kWh/h) / Power consumption of water heater per hour at time $ t $ on date $ d $    
$E_{E,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の消費電力量(kWh/h) / Power consumption of household appliances per hour at time $ t $ of date $ d $    
$E_{E,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の消費電力量(kWh/h) / Cooking power consumption per hour at time $ t $ on date $ d $    
$E_{E,PV,h,d,t}$：日付$d$の時刻$t$における1 時間当たりの太陽光発電設備による発電量のうちの自家消費分(kWh/h) / Of the amount of power generated by the photovoltaic power generation facility per hour at time $ t $ on date $ d $    
$E_{E,CG,h,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備による発電量のうちの自家消費分(kWh/h) / Of the power generated by the cogeneration facility per hour at time $ t $ on date $ d $   

1 年当たりの設計消費電力量$E_{E}$は、数値に小数点以下一位未満の端数があるときは、これを四捨五入する。 

Design power consumption $ E_ {E} $ per year is rounded off if the number has a fraction less than one decimal place.

In [ ]:
# 1 年当たりの設計消費電力量（kWh/年）
def get_E_E(region, sol_region, A_A, A_MR, A_OR, HW, Q, TS, mu_H, mu_C, r_A_ufvnt, underfloor_insulation,
            NV_MR, NV_OR, mode_H, mode_C,
            V, L,
            H_A=None,
            H_MR=None, H_OR=None, H_HS=None,
            CG=None,
            SHC=None,
            L_H_A_d_t=None, L_T_H_d_t_i=None,
            C_A=None, C_MR=None, C_OR=None, L_CS_A_d_t=None, L_CL_A_d_t=None,
            L_CS_d_t=None, L_CL_d_t=None,
            HEX=None, PV=None, solrad=None
            ):
    # 想定人数
    n_p = get_n_p(A_A)

    # 実質的な暖房機器の仕様を取得
    spec_MR, spec_OR = get_virtual_heating_devices(region, H_MR, H_OR)

    # 実質的な温水暖房機の仕様を取得
    spec_HS = get_virtual_heatsource(region, H_HS)

    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = get_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)

    # その他または設置しない場合
    spec_HW = get_virtual_hotwater(region, HW)

    # 温水暖房負荷の計算
    L_HWH = calc_L_HWH(A_A, A_MR, A_OR, HEX, H_HS, H_MR, H_OR, Q, SHC, TS, mu_H, r_A_ufvnt, region, sol_region,
                       underfloor_insulation, CG)

    # 暖房日の計算
    heating_flag_d = calc_heating_flag_d(A_A, A_MR, A_OR, HEX, H_MR, H_OR, Q, SHC, TS, mu_H, r_A_ufvnt, region,
                                         sol_region, underfloor_insulation)

    # 暖房設備の消費電力量
    E_E_H_d_t = get_E_E_H_d_t(region, sol_region, A_A, A_MR, A_OR,
                              H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, SHC, heating_flag_d, L_H_A_d_t,
                              L_T_H_d_t_i)

    # 1時間当たりの冷房設備の消費電力量
    E_E_C_d_t = get_E_E_C_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)

    # 1 時間当たりの機械換気設備の消費電力量
    E_E_V_d_t = get_E_E_V_d_t(n_p, A_A, V, HEX)

    # 1 時間当たりの照明設備の消費電力量
    E_E_L_d_t = get_E_E_L_d_t(n_p, A_A, A_MR, A_OR, L)

    # 1日当たりの給湯設備の消費電力量
    E_E_W_d_t = get_E_E_W_d_t(n_p, L_HWH, heating_flag_d, region, sol_region, spec_HW, SHC)

    # 1 時間当たりの家電の消費電力量
    E_E_AP_d_t = get_E_E_AP_d_t(n_p)

    # 1 時間当たりの調理の消費電力量
    E_E_CC_d_t = get_E_E_CC_d_t()

    # 1時間当たりのコージェネレーション設備による発電量のうちの自家消費分 (kWh/h)
    E_CG, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_TU_aux_d_t, E_E_CG_h_d_t, E_G_CG_ded, e_BB_ave, Q_CG_h =\
        get_E_W(A_A, region, sol_region, HW, SHC, CG, H_A, H_MR, H_OR, H_HS, C_A, C_MR,
                C_OR,
                                V, L, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS,
                                r_A_ufvnt, HEX, underfloor_insulation, mode_H, mode_C)

    # 太陽光発電設備の発電量
    E_E_PV_d_t = get_E_E_PV_d_t(PV, solrad)
    E_E_dmd_d_t = get_E_E_dmd_d_t(E_E_H_d_t, E_E_C_d_t, E_E_V_d_t, E_E_L_d_t, E_E_W_d_t, E_E_AP_d_t)
    E_E_PV_h_d_t = get_E_E_PV_h_d_t(E_E_PV_d_t, E_E_dmd_d_t, E_E_CG_h_d_t, has_PV=PV != None)

    E_E = sum(E_E_H_d_t) \
          + sum(E_E_C_d_t) \
          + sum(E_E_V_d_t) \
          + sum(E_E_L_d_t) \
          + sum(E_E_W_d_t) \
          + sum(E_E_AP_d_t) \
          + sum(E_E_CC_d_t) \
          - sum(E_E_PV_h_d_t) \
          - sum(E_E_CG_h_d_t)

    # 小数点以下一位未満の端数があるときは、これを四捨五入する。
    return Decimal(E_E).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP), E_E_PV_h_d_t, E_E_PV_d_t, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_dmd_d_t, E_E_TU_aux_d_t

$$
\displaystyle E_{G} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,H,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,C,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,W,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,CG,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,AP,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{G,CC,d,t} }} - E_{G,CG,sell}
$$

$E_{G}$：1 年当たりの設計ガス消費量(MJ/年) / Design gas consumption per year   
$E_{G,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの暖房設備のガス消費量(MJ/h) / Gas consumption of heating equipment per hour at time $ t $ on date $ d $    
$E_{G,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの冷房設備のガス消費量(MJ/h) / Cooling system gas consumption per hour at time $ t $ on date $ d $    
$E_{G,W,d,t}$：日付$d$の時刻$t$における1 時間当たりの給湯設備のガス消費量(MJ/h) / Gas consumption of water heater per hour at time $ t $ on date $ d $    
$E_{G,CG,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備のガス消費量(MJ/h) / Gas consumption of cogeneration facility per hour at time $ t $ of date $ d $   
$E_{G,CG,sell}$：1 年当たりのコージェネレーション設備による売電量に係るガス消費量の控除量(MJ/年) / Deduction of gas consumption amount related to power sales volume by cogeneration facility per year   
$E_{G,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電のガス消費量(MJ/h) / Gas consumption of household appliances per hour at time $ t $ on date $ d $    
$E_{G,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理のガス消費量(MJ/h) / Cooking gas consumption per hour at time $ t $ on date $ d $    

1 年当たりの設計ガス消費量$E_{G}$は、数値に小数点以下一位未満の端数があるときは、これを四捨五入する。 

Design gas consumption $ E_ {G} $ per year is rounded off if the number has a fraction less than one decimal place.

In [ ]:
# 1 年当たりの設計ガス消費量（MJ/年）
def get_E_G(region, sol_region, A_A, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS, r_A_ufvnt, HEX, underfloor_insulation,
            H_A, H_MR, H_OR, H_HS, C_A, C_MR, C_OR, V, L, HW, SHC,
            spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, mode_H, mode_C, CG, L_T_H_d_t_i,
            L_HWH, heating_flag_d):
    # 仮想居住人数
    n_p = get_n_p(A_A)

    # 暖房設備のガス消費量（MJ/h）
    E_G_H_d_t = get_E_G_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                              L_T_H_d_t_i)

    # 1時間当たりの冷房設備のガス消費量
    E_G_C_d_t = get_E_G_C_d_t()

    # 1日当たりの給湯設備のガス消費量 (MJ/d)
    E_G_W_d = get_E_G_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, HW, SHC)

    # 1日当たりのコージェネレーション設備のガス消費量
    E_G_CG_d_t, *args = calc_E_CG_d_t(A_A, region, sol_region, HW, SHC, CG, H_A, H_MR, H_OR, H_HS, C_A, C_MR,
                           C_OR,
                           V, L, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS,
                           r_A_ufvnt, HEX, underfloor_insulation, mode_H, mode_C)

    # 1 時間当たりの家電のガス消費量
    E_G_AP_d_t = get_E_G_AP_d_t()

    # 1 時間当たりの調理のガス消費量
    E_G_CC_d_t = get_E_G_CC_d_t(n_p)

    E_G = sum(E_G_H_d_t) \
          + sum(E_G_C_d_t) \
          + sum(E_G_W_d) \
          + sum(E_G_CG_d_t) \
          + sum(E_G_AP_d_t) \
          + sum(E_G_CC_d_t) 

    # 小数点以下一位未満の端数があるときは、これを四捨五入する。
    return Decimal(E_G).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)

$$
\displaystyle E_{K} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,H,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,C,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,W,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,CG,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,AP,d,t} }} + \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{K,CC,d,t} }}
$$

$E_{K}$：1 年当たりの設計灯油消費量(MJ/年) / Design kerosene consumption per year   
$E_{K,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの暖房設備の灯油消費量(MJ/h) / Kerosene consumption of heating system per hour at time $ t $ on date $ d $    
$E_{K,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの冷房設備の灯油消費量(MJ/h) / Kerosene consumption of cooling system per hour at time $ t $ of date $ d $    
$E_{K,W,d,t}$：日付$d$の時刻$t$における1 時間当たりの給湯設備の灯油消費量(MJ/h) / Consumption of kerosene in hot water supply equipment per hour at time $ t $ on date $ d $    
$E_{K,CG,d,t}$：日付$d$の時刻$t$における1 時間当たりのコージェネレーション設備の灯油消費量(MJ/h) / Kerosene consumption of cogeneration equipment per hour at time $ t $ on date $ d $    
$E_{K,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の灯油消費量(MJ/h) / Household appliances kerosene consumption per hour at time $ t $ on date $ d $    
$E_{K,CC,d,t}$：日付$d$の時刻$t$における1 時間当たりの調理の灯油消費量(MJ/h) / Cooking kerosene consumption per hour at time $ t $ on date $ d $    

1 年当たりの設計灯油消費量$E_{K}$は、数値に小数点以下一位未満の端数があるときは、これを四捨五入する。 

Design kerosene consumption $ E_ {K} $ per year is rounded off if the number has a fraction less than one decimal place.

In [ ]:
# 1 年当たりの設計灯油消費量（MJ/年）
def get_E_K(region, sol_region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t_i,
            L_HWH, heating_flag_d, HW, SHC):
    # 仮想居住人数
    n_p = get_n_p(A_A)

    # 暖房設備の灯油消費量（MJ/h）
    E_K_H_d_t = get_E_K_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                              L_T_H_d_t_i)

    # 1時間当たりの冷房設備の灯油消費量
    E_K_C_d_t = get_E_K_C_d_t()

    # 1日当たりの給湯設備の灯油消費量 (MJ/d)
    E_K_W_d_t = get_E_K_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, HW, SHC)

    # 1日当たりのコージェネレーション設備の灯油消費量
    E_K_CG_d = np.zeros(365)

    # 1 時間当たりの家電の灯油消費量
    E_K_AP_d_t = get_E_K_AP_d_t()

    # 1 時間当たりの調理の灯油消費量
    E_K_CC_d_t = get_E_K_CC_d_t()

    # 1年当たりの設計灯油消費量（MJ / 年）
    E_K_raw = sum(E_K_H_d_t) \
              + sum(E_K_C_d_t) \
              + sum(E_K_W_d_t) \
              + sum(E_K_CG_d) \
              + sum(E_K_AP_d_t) \
              + sum(E_K_CC_d_t)

    # 小数点以下一位未満の端数があるときは、これを四捨五入する。
    E_K = Decimal(E_K_raw).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)

    return E_K

$$
\large E_{UT,H} = \sum_{d=1}^{365}{\sum_{t=0}^{23}{E_{UT,H,d,t} }}
$$

$E_{UT,H}$：1 年当たりの未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/年) / Design primary energy consumption equivalent of raw heating load per year   
$E_{UT,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h) / Designing the primary energy consumption equivalent of the raw heating load of the heating installation per hour at the time $ t $ of the date $ d $    

1 年当たりの未処理暖房負荷の設計一次エネルギー消費量相当値$E_{UT,H}$は、数値に小数点以下一位未満の端数があるときは、これを四捨五入する。 

Designing Primary Energy Consumption Equivalents $ E_ {UT, H} $ of Raw Heating Load per Year If the number has a fraction less than one decimal place, round it off.

In [ ]:
# 1 年当たりの未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/年）
def get_E_UT_H(region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_H_A_d_t,
               L_T_H_d_t):
    # 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値
    E_UT_H_d_t = get_E_UT_H_d_t(region, A_A, A_MR, A_OR, mode_H, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                                L_H_A_d_t,
                                L_T_H_d_t)

    # 1 年当たりの未処理暖房負荷の設計一次エネルギー消費量相当値
    E_UT_H_raw = sum(E_UT_H_d_t)

    # 小数点以下一位未満の端数があるときは、これを四捨五入する。
    E_UT_H = Decimal(E_UT_H_raw).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)

    return E_UT_H

### 14. 各設備のエネルギー消費量等 / Energy consumption of each facility

省略

Omission

### 15. 居住人数 / Number of residents

省略

Omission

### 16. 電力需要 / Power demand

$$
\large 
E_{E,dmd,d,t} = E_{E,H,d,t} + E_{E,C,d,t} + E_{E,V,d,t} + E_{E,L,d,t} + E_{E,W,d,t} + E_{E,AP,d,t}
$$

$E_{E,dmd,d,t}$：日付$d$の時刻$t$における1 時間当たりの電力需要(kWh/h) / Power demand per hour at time $ t $ on date $ d $    
$E_{E,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの暖房設備の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ of date $ d $    
$E_{E,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの冷房設備の消費電力量(kWh/h) / Power consumption of cooling system per hour at time $ t $ of date $ d $    
$E_{E,V,d,t}$：日付$d$の時刻$t$における1 時間当たりの機械換気設備の消費電力量(kWh/h) / Power consumption of mechanical ventilation equipment per hour at time $ t $ of date $ d $    
$E_{E,L,d,t}$：日付$d$の時刻$t$における1 時間当たりの照明設備の消費電力量(kWh/h) / Power consumption of lighting fixtures per hour at time $ t $ on date $ d $    
$E_{E,W,d,t}$：日付$d$の時刻$t$における1 時間当たりの給湯設備の消費電力量(kWh/h) / Power consumption of water heater per hour at time $ t $ on date $ d $    
$E_{E,AP,d,t}$：日付$d$の時刻$t$における1 時間当たりの家電の消費電力量(kWh/h) / Power consumption of household appliances per hour at time $ t $ of date $ d $    

In [ ]:
# 1時間当たりの電力需要
def get_E_E_dmd_d_t(E_E_H_d_t, E_E_C_d_t, E_E_V_d_t, E_E_L_d_t, E_E_W_d_t, E_E_AP_d_t):
    """
    :param E_E_H_d_t: 1時間当たりの暖房設備の消費電力量 (kWh/h)
    :param E_E_C_d_t: 1時間当たりの冷房設備の消費電力量 (kWh/h)
    :param E_E_V_d_t: 1時間当たりの機械換気設備の消費電力量 (kWh/h)
    :param E_E_L_d_t: 1時間当たりの照明設備の消費電力量 (kWh/h)
    :param E_E_W_d_t: 1時間当たりの給湯設備の消費電力量 (kWh/h)
    :param E_E_AP_d_t: 1時間当たりの家電の消費電力量 (kWh/h)
    :return: 1時間当たりの電力需要 (kWh/h)
    """
    return E_E_H_d_t + E_E_C_d_t + E_E_V_d_t + E_E_L_d_t + E_E_W_d_t + E_E_AP_d_t